Connecting to Google Drive 

In [1]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
import import_ipynb 
connect_to_drive()

#The path of the root folder bioinformatics should be inserted here in order to import all the modules
root_folder='/content/gdrive/My Drive/Bioinformatics3/bioinformatics/'
import sys
sys.path.append(root_folder)

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=63f7748f4b41646c7b7e9ed01af9f6b113d8eec9f508d99f4b2130d52716fd9c
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
Mounted at /content/gdrive


Importing modules

In [2]:
#Put this to False if you don't intend to use the wandb library to log the results
use_wandb=False
save_model=False

In [11]:
import torch
from utils.utils import *

install_requirements(use_wandb)
import albumentations as A
from torchvision import transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.augmentations.transforms import *
import segmentation_models_pytorch as smp
from torchmetrics.functional import iou as iou_torch
from tqdm import tqdm
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR

if use_wandb:
  import wandb

from Dataset.VascularDataset import *
from Dataset.VascularClassificationDataset import *

importing Jupyter notebook from /content/gdrive/My Drive/Bioinformatics3/bioinformatics/Dataset/VascularClassificationDataset.ipynb


Retrieve images paths from txt

In [7]:
#Retrieving images from saved_predictions folder
import os
root_path=root_folder+'Predictions/'

training_path=root_path+'Training/'
validation_path=root_path+'Validation/'
test_path=root_path+'Test/'

saved_predictions_training = [training_path+path for path in os.listdir(training_path)]
saved_predictions_validation=[validation_path+path for path in os.listdir(validation_path)]
saved_predictions_test=[test_path+path for path in os.listdir(test_path)]

Defining model, loss function, metrics, and initializing Wandb

In [20]:
from torch.nn import BCELoss
from torch.nn import BCEWithLogitsLoss
from torchvision.models import resnet50

batch_size=20
lr=0.01
momentum=0.9
max_n_epochs=80
dataset_type='original'
step_size=5
size=224
gamma=0.8

model = resnet50(num_classes=2)
model.cuda()

optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

loss=BCELoss
scheduler=StepLR(optimizer, step_size=step_size, gamma=gamma)

if use_wandb:
  wandb.init(project="Bioinformatics_classification_experiments_original",
            config={
                "batch_size": batch_size,
                "model": model,
                "learning_rate": lr,
                "max_n_epochs": max_n_epochs,
                "momentum": momentum,
                "optimizer": optimizer,
                "dataset": dataset_type,
                "loss": loss,
                "transforms": transforms,
                "scheduler": scheduler,
                "step_size":step_size,
                "gamma":gamma,
                "size": size
            })

  run_id=wandb.run.id

if save_model:
  save_model_path=save_model_folder+run_id

Defining dataset and dataloaders

In [19]:
training_dataset=VascularClassificationDataset(saved_predictions_training, size=size)
validation_dataset=VascularClassificationDataset(saved_predictions_training, size=size)
test_dataset=VascularClassificationDataset(saved_predictions_test, size=size)

train_dataloader=DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_dataloader=DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Training, validation and logging into Wandb

In [ ]:
from torch.nn.functional import binary_cross_entropy
from torch.nn import Sigmoid
from sklearn.metrics import f1_score
from torch.nn import CrossEntropyLoss

criterion=CrossEntropyLoss()
highest_f1=0

for epoch in range(1, max_n_epochs+1, 1):

  model.train()
  training_loss=0
  training_f1=0

  y_true=[]
  y_pred=[]

  for x,y in tqdm(train_dataloader): 

    optimizer.zero_grad()
    out=model((x>0).cuda().float())
    my_loss=criterion(out.cuda(), y.cuda())

    for tupl in out:
      if tupl[0]>tupl[1]:
        y_pred.append(0)
      else:
        y_pred.append(1)
    for target in y:
      y_true.append(target)

    training_loss+=my_loss.item()
    my_loss.backward()
    optimizer.step()
  
  training_f1=f1_score(y_pred, y_true)
  
  training_samples=len(y_true)
  training_loss=training_loss/len(y_true)

  print(f'Epoch #{epoch}, training loss: {training_loss/len(y_true)}, training f1: {training_f1}')

  scheduler.step()

  model.eval()
  validation_loss=0
  validation_iou=0
  y_true=[]
  y_pred=[]

  for x,y in tqdm(valid_dataloader): 

    out=model((x>0).cuda().float())
    my_loss=criterion(out, y.cuda()) 

    for tupl in out:
      if tupl[0]>tupl[1]:
        y_pred.append(0)
      else:
        y_pred.append(1)
    for target in y:
      y_true.append(target)

    validation_loss+=my_loss.item()

  validation_f1=f1_score(y_pred, y_true)

  validation_samples=len(y_true)
  validation_loss=validation_loss/validation_samples

  print(f'Epoch #{epoch}, validation loss: {validation_loss}, validation f1: {validation_f1}')

  if use_wandb:
    wandb.log({"training loss": training_loss, "validation loss": validation_loss, "epoch":epoch})
    wandb.log({"training iou": training_f1, "validation iou": validation_f1, "epoch": epoch})

  if validation_f1>highest_f1:
    last_epoch_saved=epoch
    highest_f1=validation_f1

    if save_model:  
      print(f'saving model into {save_model_path}')
      torch.save(model,f'{save_model_path}.pth')

100%|██████████| 8/8 [00:03<00:00,  2.06it/s]


Epoch #1, training loss: 0.0010099722561426462, training f1: 0.5217391304347826


100%|██████████| 8/8 [00:01<00:00,  4.97it/s]


Epoch #1, validation loss: 90.37755584716797, validation f1: 0.0


100%|██████████| 8/8 [00:03<00:00,  2.14it/s]


...

Epoch #16, training loss: 2.0699366008614106e-06, training f1: 0.9938650306748467


100%|██████████| 8/8 [00:01<00:00,  4.95it/s]


Epoch #16, validation loss: 0.00031943423482516664, validation f1: 0.993939393939394


 25%|██▌       | 2/8 [00:01<00:03,  1.69it/s]


KeyboardInterrupt: ignored

TESTING

In [ ]:
#loading model
from torch.nn import Sigmoid
model_path='/content/gdrive/MyDrive/Bioinformatics2/saved_models/1o9z0tra.pth'
model=torch.load(model_path, map_location=torch.device('cuda'))

In [ ]:
 model.eval()

test_loss=0
test_f1=0
y_true=[]
y_pred=[]

for x,y in tqdm(test_dataloader): 

  out=model((x>0).cuda().float())
  my_loss=criterion(out, y.cuda()) 

  for tupl in out:
    if tupl[0]>tupl[1]:
      y_pred.append(0)
    else:
      y_pred.append(1)
  for target in y:
    y_true.append(target)

  test_loss+=my_loss.item()

test_f1=f1_score(y_pred, y_true)

test_samples=len(y_true)
test_loss=test_loss/validation_samples

print(f'Epoch #{epoch}, test loss: {test_loss}, test f1: {test_f1}')

print([(pred,target) for pred, target in zip(y_pred,y_true)])

100%|██████████| 1/1 [00:00<00:00,  3.41it/s]

Epoch #17, test loss: 1.4927728443581146e-06, test f1: 1.0
[(1, tensor(1)), (1, tensor(1)), (1, tensor(1)), (0, tensor(0)), (1, tensor(1)), (0, tensor(0)), (0, tensor(0)), (0, tensor(0)), (1, tensor(1)), (1, tensor(1))]
